### Notatki
- można zapisywać przez save=true,
- results[0].boxes - informacje o bounding boxach,
- ...boxes[0].data -> [x_topleft, y_topleft, x_bottomright, y_bottomright, conf, class]

In [1]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [26]:
model = YOLO("checkpoints/YOLO11/yolo11m.pt")

results = model.predict('images/weti1_maska.jpg', save=True, classes=[2], conf=0.1) # 2 - car


image 1/1 /mnt/c/Users/gtraw/Projekt_badawczy/images/weti1_maska.jpg: 384x640 13 cars, 63.9ms
Speed: 4.0ms preprocess, 63.9ms inference, 7.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict6


In [17]:
results[0].show() # pepeW

Opening "/tmp/tmp_hs29riq.PNG" with ImageMagick (color depth=q16)  (image/png)


In [34]:
for b in results[0].boxes:
    print(b.data)

tensor([[421.7600, 529.3541, 476.7028, 592.8027,   0.7753,   2.0000]], device='cuda:0')
tensor([[4.2860e+02, 8.1133e+02, 4.9102e+02, 8.8302e+02, 7.1264e-01, 2.0000e+00]], device='cuda:0')
tensor([[4.8694e+02, 5.1997e+02, 5.3437e+02, 5.9040e+02, 5.8331e-01, 2.0000e+00]], device='cuda:0')
tensor([[4.4853e+02, 6.6903e+02, 5.1021e+02, 7.4299e+02, 5.6091e-01, 2.0000e+00]], device='cuda:0')
tensor([[4.0432e+02, 6.2024e+02, 4.4572e+02, 7.4502e+02, 5.4591e-01, 2.0000e+00]], device='cuda:0')
tensor([[4.6189e+02, 6.3232e+02, 5.1668e+02, 6.7655e+02, 5.3744e-01, 2.0000e+00]], device='cuda:0')
tensor([[4.8696e+02, 3.8527e+02, 5.2406e+02, 4.1938e+02, 4.9034e-01, 2.0000e+00]], device='cuda:0')
tensor([[3.9789e+02, 3.9382e+02, 4.3199e+02, 4.2319e+02, 4.2813e-01, 2.0000e+00]], device='cuda:0')
tensor([[4.4882e+02, 6.3082e+02, 5.1486e+02, 7.4377e+02, 3.9271e-01, 2.0000e+00]], device='cuda:0')
tensor([[5.3269e+02, 3.8091e+02, 5.7162e+02, 4.1561e+02, 3.2380e-01, 2.0000e+00]], device='cuda:0')
tensor([[4.3

In [29]:
for i, r in enumerate(results[0]):
    print(i, r.keypoints())

TypeError: 'NoneType' object is not callable

### Segmentacja na całym zbiorze - średnia

In [16]:
from ultralytics import YOLO

import cv2
import sys
import os
import time
import json
import numpy as np
from shapely.geometry import Polygon
from scipy.optimize import linear_sum_assignment

model_checkpoints = {
    "yolo11m": "checkpoints/YOLO11/yolo11m.pt",
    "yolo11n": "checkpoints/YOLO11/yolo11n.pt",
    "yolo11s": "checkpoints/YOLO11/yolo11s.pt",
    "yolo11l": "checkpoints/YOLO11/yolo11l.pt",
    "yolo11x": "checkpoints/YOLO11/yolo11x.pt"
}

input_dir = '/mnt/c/Users/gtraw/Projekt_badawczy/dataset_11_14'
output_dir = '/mnt/c/Users/gtraw/Projekt_badawczy/dataset_results_11_14/ious'

In [ ]:
def transform_to_polygons(data, mode="points"):
    polygons = []
    if mode == "points":
        polygons = [points2polygon(d) for d in data]
    elif mode == "corners":
        polygons = [corners2polygon(d) for d in data]
    elif mode == "mask":
        polygons = [mask2polygon(d) for d in data]
    return [p for p in polygons if p is not None]
    
def corners2polygon(corners):
    x1, y1, x2, y2 = corners
    points = [(x1, y1), (x2, y1), (x2, y2), (x1, y2), (x1, y1)]
    
    return Polygon(points)
        
def points2polygon(points):
    return Polygon(points)

def mask2polygon(mask):
    mask = np.array(mask)
    mask = mask.astype(np.uint8)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None
    
    largest_contour = max(contours, key=cv2.contourArea)
    
    polygon = Polygon(shell=largest_contour.squeeze())
    
    if not polygon.is_valid:
        polygon = polygon.buffer(0)
    
    return polygon

def iou(polygon1, polygon2):
    if not polygon1.is_valid:
        polygon1 = polygon1.buffer(0)
    if not polygon2.is_valid:
        polygon2 = polygon2.buffer(0)
    union = polygon1.union(polygon2).area
    if union <= 0:
        return 0
    intersection = polygon1.intersection(polygon2).area
    return intersection / union

def build_similarity_matrix(objects_db, objects_pred):
    len_db, len_pred = len(objects_db), len(objects_pred)
    print(f"Matrix size: [{len_db}, {len_pred}]")
    similarity_matrix = np.zeros([len_db, len_pred])
    
    time_start = time.time()
    for x, obj_db in enumerate(objects_db):
        for y, obj_pred in enumerate(objects_pred):
            similarity_matrix[x, y] = iou(obj_db, obj_pred)
            print(f"Building similarity matrix: {x * len_pred + y + 1:>10}/{len_db*len_pred}", end='\r')
    time_end = time.time() - time_start
    print(f"Similarity matrix built in {time_end:.2f}s" + ' ' * 100)
    return similarity_matrix

def match_polygons(similarity_matrix):
    xs, ys = linear_sum_assignment(similarity_matrix, maximize=True)
    matching = [[x, y, similarity_matrix[x, y]] for x, y in zip(xs, ys) if similarity_matrix[x][y] > 0]
    return matching

def evaluate_segmentation(polygons_db, polygons_pred, matching, time, weather_info = [], model_info = []):
    results = {
        "weather": weather_info,
        "model": model_info,
        "time": time
    }
    num_db = len(polygons_db)
    num_pred = len(polygons_pred)
    num_matching = len(matching) 

    results['ground_truth_objects'] = num_db
    results['segmented_objects'] = num_pred
    results['matched_objects'] = num_matching

    if num_pred != 0:
        results['precision'] = num_matching / num_pred
    else:
        results['precision'] = 0

    if num_db != 0:
        results['recall'] = num_matching / num_db
    else:
        results['recall'] = 0

    if results['precision'] + results['recall'] != 0:
        results['f1'] = 2 * results['precision'] * results['recall'] / (results['precision'] + results['recall'])
    else:
        results['precision'] = 0

    ious = []
    for x, y, _ in matching:
        ious.append(iou(polygons_db[x], polygons_pred[y]))

    if num_matching != 0:
        results['mean_iou'] = sum(ious) / num_matching
    else:
        results['mean_iou'] = 0

    for i in range(num_db - num_matching):
        ious.append(0)

    return results, ious

def evaluate_individually(polygons_db, polygons_pred, matching):
    num_db = len(polygons_db)
    num_pred = len(polygons_pred)
    num_matching = len(matching)

    ious = []
    for x, y, _ in matching:
        ious.append(iou(polygons_db[x], polygons_pred[y]))
    for i in range(num_db - num_matching):
        ious.append(0)

    return ious

#### Uśrednione i IoU

In [ ]:
for model_name in model_checkpoints.keys():
    full_iou = {
        'model': [],
        'model_params': [],
        'filename': [],
        'iou': []
    }
    model = YOLO(model_checkpoints[model_name])
    output_dir_model = f"{output_dir}_{model_name}"
    print(f"output_dir: {output_dir_model}")

    # fix directories
    if not os.path.exists(output_dir_model):
        os.makedirs(output_dir_model)

    # for image in dir
    for image_file in os.listdir(input_dir):
        data_pred = {
            'model': model_name
        }

        # skip non-images
        if not image_file.endswith(('jpg', 'png')):
            continue

        print(f"\nImage: {image_file}")
        # image's path
        image_path = os.path.join(input_dir, image_file)
        # measure time
        time_start = time.time()
        # generate masks
        results = model.predict(image_path, classes=[2])
        time_end = time.time() - time_start

        # save results
        data_pred['time'] = time_end
        data_pred['objects'] = []
        for bbox in results[0].boxes:
            data_pred['objects'].append(bbox.data.cpu().numpy()[0][:4])

        # load labeled data
        data_filename = '.'.join(image_file.replace('_m', '').split('.')[:-1]) + '.json'
        with open(os.path.join(input_dir, data_filename), 'r') as f:
            data_db = json.load(f)

        data_db['polygons'] = transform_to_polygons([obj['points'] for obj in data_db['objects']])
        data_pred['polygons'] = transform_to_polygons([obj for obj in data_pred['objects']], "corners")

        sm = build_similarity_matrix(data_db['polygons'], data_pred['polygons'])
        matching = match_polygons(sm)

        # save mean results
        result_dict, iou_results = evaluate_segmentation(data_db['polygons'], data_pred['polygons'], matching, data_pred['time'], data_db['weather'], ['sam2.1', model_name])
        result_dict['filename'] = '.'.join(image_file.split('.')[:-1])

        json_path = os.path.join(output_dir_model, f"{'.'.join(image_file.split('.')[:-1])}.json")
        with open(json_path, 'w') as json_file:
            json.dump(result_dict, json_file, indent=4)

        # save iou results]
        for iou_result in iou_results:
            full_iou['iou'].append(iou_result)
            full_iou['filename'].append('.'.join(image_file.split('.')[:-1]))

    for _ in full_iou['iou']:
        full_iou['model'].append(model_name)
        full_iou['model_params'].append('')

    json_path = os.path.join(f"{output_dir_model}/ious" f"{model_name}_iou.json")
    with open(json_path, 'w') as json_file:
        json.dump(full_iou, json_file, indent=4)


#### Tylko IoU

In [ ]:

output_dir = '/mnt/c/Users/gtraw/Projekt_badawczy/dataset_results_11_14/ious'

In [ ]:
for model_name in model_checkpoints.keys():
    full_iou = {
        'model': [],
        'model_params': [],
        'filename': [],
        'iou': []
    }
    model = YOLO(model_checkpoints[model_name])
    output_dir_model = f"{output_dir}" #f"{output_dir}_{model_name}"
    print(f"output_dir: {output_dir_model}")

    # fix directories
    if not os.path.exists(output_dir_model):
        os.makedirs(output_dir_model)

    # for image in dir
    for image_file in os.listdir(input_dir):
        data_pred = {
            'model': model_name
        }

        # skip non-images
        if not image_file.endswith(('jpg', 'png')):
            continue

        print(f"\nImage: {image_file}")
        # image's path
        image_path = os.path.join(input_dir, image_file)
        # measure time
        time_start = time.time()
        # generate masks
        results = model.predict(image_path, classes=[2])
        time_end = time.time() - time_start

        # save results
        data_pred['time'] = time_end
        data_pred['objects'] = []
        for bbox in results[0].boxes:
            data_pred['objects'].append(bbox.data.cpu().numpy()[0][:4])

        # load labeled data
        data_filename = '.'.join(image_file.replace('_m', '').split('.')[:-1]) + '.json'
        with open(os.path.join(input_dir, data_filename), 'r') as f:
            data_db = json.load(f)

        data_db['polygons'] = transform_to_polygons([obj['points'] for obj in data_db['objects']])
        data_pred['polygons'] = transform_to_polygons([obj for obj in data_pred['objects']], "corners")

        sm = build_similarity_matrix(data_db['polygons'], data_pred['polygons'])
        matching = match_polygons(sm)

        # save mean results
        # result_dict = evaluate_segmentation(data_db['polygons'], data_pred['polygons'], matching, data_pred['time'], data_db['weather'], ['sam2.1', model_name])
        # result_dict['filename'] = '.'.join(image_file.split('.')[:-1])

        # json_path = os.path.join(output_dir_model, f"{'.'.join(image_file.split('.')[:-1])}.json")
        # with open(json_path, 'w') as json_file:
        #     json.dump(result_dict, json_file, indent=4)

        # save iou results]
        for iou_result in evaluate_individually(data_db['polygons'], data_pred['polygons'], matching):
            full_iou['iou'].append(iou_result)
            full_iou['filename'].append('.'.join(image_file.split('.')[:-1]))

    for _ in full_iou['iou']:
        full_iou['model'].append(model_name)
        full_iou['model_params'].append('')

    json_path = os.path.join(output_dir_model, f"{model_name}_iou.json")
    with open(json_path, 'w') as json_file:
        json.dump(full_iou, json_file, indent=4)


output_dir: /mnt/c/Users/gtraw/Projekt_badawczy/dataset_results_11_14/ious

Image: weti1_2024_11_14_12_47.jpg

image 1/1 /mnt/c/Users/gtraw/Projekt_badawczy/dataset_11_14/weti1_2024_11_14_12_47.jpg: 384x640 21 cars, 67.8ms
Speed: 2.4ms preprocess, 67.8ms inference, 5.9ms postprocess per image at shape (1, 3, 384, 640)
Matrix size: [14, 21]
Similarity matrix built in 0.02s                                                                                                    

Image: weti1_2024_11_14_12_47_m.jpg

image 1/1 /mnt/c/Users/gtraw/Projekt_badawczy/dataset_11_14/weti1_2024_11_14_12_47_m.jpg: 384x640 9 cars, 23.1ms
Speed: 3.2ms preprocess, 23.1ms inference, 6.3ms postprocess per image at shape (1, 3, 384, 640)
Matrix size: [14, 9]
Similarity matrix built in 0.01s                                                                                                    

Image: weti1_2024_11_14_12_57.jpg

image 1/1 /mnt/c/Users/gtraw/Projekt_badawczy/dataset_11_14/weti1_2024_11_14_12_57.jpg:

### Zapisywanie bounding boxów i czasów do pliku

In [1]:
from ultralytics import YOLO

import cv2
import sys
import os
import time
import json
import numpy as np
from shapely.geometry import Polygon
from scipy.optimize import linear_sum_assignment

model_checkpoints = {
    "yolo11m": "checkpoints/YOLO11/yolo11m.pt",
    "yolo11n": "checkpoints/YOLO11/yolo11n.pt",
    "yolo11s": "checkpoints/YOLO11/yolo11s.pt",
    "yolo11l": "checkpoints/YOLO11/yolo11l.pt",
    "yolo11x": "checkpoints/YOLO11/yolo11x.pt"
}

input_dir = '/mnt/c/Users/gtraw/Projekt_badawczy/dataset_11_14'
output_dir = '/mnt/c/Users/gtraw/Projekt_badawczy/dataset_results_11_14/yolo_bbox'

In [2]:
model_name = "yolo11x"
model = YOLO(model_checkpoints[model_name])

In [ ]:
for conf in [0.01, 0.05, 0.1, 0.5]:
    # fix directories
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # for image in dir
    for image_file in os.listdir(input_dir):
        data_pred = {
            'model': model_name,
            'model_params': f"conf={conf}"
        }

        # skip non-images
        if not image_file.endswith(('jpg', 'png')):
            continue

        print(f"\nImage: {image_file}")
        # image's path
        image_path = os.path.join(input_dir, image_file)
        # measure time
        time_start = time.time()
        # generate masks
        results = model.predict(image_path, classes=[2])
        time_end = time.time() - time_start

        # save results
        data_pred['time'] = time_end
        data_pred['objects'] = []
        for bbox in results[0].boxes:
            data_pred['objects'].append([float(x) for x in bbox.data.cpu().numpy()[0][:4]])

        json_path = os.path.join(output_dir, f"yolo11x_{str(conf).split('.')[-1]}_{'.'.join(image_file.split('.')[:-1])}.json")
        with open(json_path, 'w+') as json_file:
            print(f"{data_pred=}")
            json.dump(data_pred, json_file, indent=4)